In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np
import os
import csv
from tqdm import tqdm

In [3]:
poi_path = r"/data-sat-img/postprocessed/mujiang/POI/全国POI数据/"
save_path = "/data-sat-img/postprocessed/mujiang/master-thesis/poi_summary/"
city_path = r"/data-sat-img/postprocessed/mujiang/POI/cities/"

In [4]:
geomaster_path = "/data-sat-img/postprocessed/mujiang/geobound_master/"
geomaster = pd.read_csv(geomaster_path + "lookup_table.csv")

geomaster_selected = geomaster[["MID_m","地级","省级"]]
geomaster_selected = geomaster_selected.drop_duplicates(subset=['MID_m', '地级'])
print(geomaster_selected.shape)
geomaster_selected

(416, 3)


,MID_m,地级,省级
0,1,三沙市,海南省
2,2,锡林郭勒盟,内蒙古自治区
3,3,阿坝藏族羌族自治州,四川省
4,4,哈尔滨市,黑龙江省
5,5,兴安盟,内蒙古自治区
...,...,...,...
3326,412,固原地区,宁夏回族自治区
3327,413,昭通地区,云南省
3339,414,商洛地区,陕西省
3346,415,武威地区,甘肃省


In [5]:
def find_poi_file(MID_city,csv_file):
    year_summary_df = pd.read_csv(save_path + csv_file)
    city_df = year_summary_df[year_summary_df["MID_m"] == MID_city]
    
    # if the city has data at that year
    if city_df.shape[0]>0:
        poi_files = city_df["path"].values[0]
        poi_files  = poi_files.split(";")
        return poi_files
    
    else:
        empty_list = []
        return empty_list
    

## define city

In [6]:
# case when only exporting for 1 city
city  = "深圳市"
MID_city = geomaster_selected.loc[geomaster_selected["地级"] == city]["MID_m"].values.min()
MID_city

71

In [7]:
MID_list = list(geomaster_selected["MID_m"])
city_list = list(geomaster_selected["地级"])

## find file and read file

In [8]:
os.chdir(r"/data-sat-img/postprocessed/mujiang/POI/全国POI数据/")

## 2022

In [13]:
for i,MID_city in enumerate(MID_list):
    city = city_list[i]
    csv_file = "summary_2022.csv"

    file_path = os.path.join(city_path,f"{MID_city}_{city}",f"2022_{MID_city}_{city}.csv")
    if os.path.exists(file_path):
        continue
        
    poi_file_list = find_poi_file(MID_city,csv_file)    
    if len(poi_file_list)>0:
        print(city,poi_file_list)        

        try:    
            poi_df = pd.read_csv(poi_file_list[0],encoding = "utf-8")
        except Exception as e:        
            poi_df = pd.read_csv(poi_file_list[0], encoding='GB2312',encoding_errors='ignore')        
        
        poi_df["class1"] = poi_df["大类"]
        poi_df["lon"] = poi_df["经度"]
        poi_df["lat"] = poi_df["纬度"]

        

        directory = os.path.dirname(file_path)
        if not os.path.exists(directory):
            # Create the directory
            os.makedirs(directory)

        poi_df.to_csv(file_path,encoding = "utf-8-sig",index = False)
    

长春市 ['2022POI/东北poi/吉林省/长春市POI数据.csv']


## 2021

In [12]:
for i,MID_city in enumerate(MID_list):
    city = city_list[i]
    csv_file = "summary_2021.csv"    
    
    file_path = os.path.join(city_path,f"{MID_city}_{city}",f"2021_{MID_city}_{city}.csv")
    if os.path.exists(file_path):
        continue

    poi_file_list = find_poi_file(MID_city,csv_file)    
    if len(poi_file_list)>0:
        print(city,poi_file_list)
        
        try:    
            poi_df = pd.read_csv(os.path.join(poi_file_list[0]),encoding = "utf-8",encoding_errors='ignore', on_bad_lines='skip', dtype={14: 'str'})
            poi_df = poi_df[['名称', '类别','经纬度','地址']]

        except Exception as e:
            #print(i,poi_city_list[i])        
            poi_df = pd.read_csv(os.path.join(poi_file_list[0]), encoding='GB2312',encoding_errors='ignore', on_bad_lines='skip', dtype={14: 'str'})
            poi_df = poi_df[['名称', '类别','经纬度','地址']]
               
        
        class_split_columns = poi_df["类别"].str.split(';', expand=True)
        poi_df["class1"] = class_split_columns[0]
        
        #correcting format error
        poi_df['经纬度'] = poi_df['经纬度'].astype(str)
        poi_df['经纬度'] = poi_df['经纬度'].apply(lambda x: x.replace('[', '').replace(']', '').replace(';', ','))
        poi_df = poi_df[poi_df['经纬度'].str.contains(',')]
        
        # get coordinates
        poi_df['lon'] = poi_df['经纬度'].apply(lambda x: x.split(",")[0])
        poi_df['lat'] = poi_df['经纬度'].apply(lambda x: x.split(",")[1])
        
    
        directory = os.path.dirname(file_path)
        if not os.path.exists(directory):
            # Create the directory
            os.makedirs(directory)

        poi_df.to_csv(file_path,encoding = "utf-8-sig",index = False)
        
    else:
        print(f"{city} not in the poi dataset")

三沙市 not in the poi dataset
阿拉善盟 not in the poi dataset
哈密市 not in the poi dataset
白沙黎族自治县 not in the poi dataset
保亭黎族苗族自治县 not in the poi dataset
北屯市 not in the poi dataset
昌江黎族自治县 not in the poi dataset
澄迈县 not in the poi dataset
张家界市 not in the poi dataset
儋州市 not in the poi dataset
定安县 not in the poi dataset
东方市 not in the poi dataset
台湾省 not in the poi dataset
鞍山市 not in the poi dataset
台州市 ['2021POI/096-台州市-482351.csv']
揭阳市 ['2021POI/214-揭阳市-223294.csv']
六安市 ['2021POI/110-六安市-236935.csv']
济宁市 ['2021POI/141-济宁市-350518.csv']
嘉峪关市 ['2021POI/297-嘉峪关-19954.csv']
普洱市 ['2021POI/269-普洱市-78873.csv']
阳江市 ['2021POI/209-阳江市-202549.csv']
金昌市 ['2021POI/298-金昌市-21276.csv']
常州市 ['2021POI/077-常州市-455312.csv']
丽水市 ['2021POI/097-丽水市-147138.csv']
固原市 ['2021POI/321-固原市-44558.csv']
西双版纳傣族自治州 ['2021POI/274-西双版纳-54596.csv']
绍兴市 ['2021POI/092-绍兴市-384473.csv']
可克达拉市 not in the poi dataset
韶关市 ['2021POI/196-韶关市-158703.csv']
乐东黎族自治县 not in the poi dataset
娄底市 ['2021POI/193-娄底市-143671.csv']
亳州市 ['2021POI/111-

### 2020

In [18]:
for i,MID_city in enumerate(MID_list):
    city = city_list[i]
    csv_file = "summary_2020.csv"

    file_path = os.path.join(city_path,f"{MID_city}_{city}",f"2020_{MID_city}_{city}.csv")
    if os.path.exists(file_path):
        continue
        
    poi_file_list = find_poi_file(MID_city,csv_file)    
    if len(poi_file_list)>0:
        print(city,len(poi_file_list))        

        poi_df = pd.read_csv(poi_file_list[0])
        poi_df_all = poi_df.copy()
        poi_df_all.drop(poi_df_all.index, inplace=True)
        
        
        
        for poi_file in poi_file_list:
            if poi_file.endswith("csv"):
                poi_df = pd.read_csv(poi_file,encoding_errors='ignore', on_bad_lines='skip')        
                poi_df_all = pd.concat([poi_df_all, poi_df])
                
        poi_df_all["class1"] = poi_df_all["大类"]
        poi_df_all["lon"] = poi_df_all["wgs84lon"]
        poi_df_all["lat"] = poi_df_all["wgs84lat"]
        
        poi_df_all = poi_df_all[['name', 'dtype', 'address', 'pname', 'cityname',  'adname','marlon', 'marlat', 'wgs84lon', 'wgs84lat', '大类', '中类',
       '小类', 'class1', 'lon', 'lat']]


        directory = os.path.dirname(file_path)
        if not os.path.exists(directory):
            # Create the directory
            os.makedirs(directory)

        poi_df_all.to_csv(file_path,encoding = "utf-8-sig",index = False)
    
    else:
        print(f"{city} not in the poi dataset")

三沙市 not in the poi dataset
阿拉尔市 not in the poi dataset
那曲市 not in the poi dataset
景德镇市 23
保山市 23
北京市 24
宝鸡市 23
阿里地区 21
白城市 23
安庆市 not in the poi dataset
百色市 22
巴彦淖尔市 23
定安县 not in the poi dataset
大庆市 not in the poi dataset
白山市 22
台湾省 not in the poi dataset
安顺市 22
北海市 22
鞍山市 not in the poi dataset
胡杨河市 not in the poi dataset
新星市 not in the poi dataset
白杨市 not in the poi dataset
哈密地区 3
山南地区 3
大兴安岭地区（加格达奇） not in the poi dataset
省直辖县级行政单位 not in the poi dataset
省直辖县级行政单位 not in the poi dataset
昌都地区 not in the poi dataset
日喀则地区 not in the poi dataset
林芝地区 not in the poi dataset
吐鲁番地区 not in the poi dataset
自治区直辖县级行政单位 not in the poi dataset
省直辖行政单位 not in the poi dataset
海东地区 not in the poi dataset
铜仁地区 not in the poi dataset
毕节地区 not in the poi dataset
巢湖市 not in the poi dataset
襄樊市 not in the poi dataset
思茅市 not in the poi dataset
吕梁地区 not in the poi dataset
吕梁地区
提醒你儿子啊，明天教历史作业
 not in the poi dataset
巴彦淖尔盟 not in the poi dataset
乌兰察布盟 not in the poi dataset
酒泉地区 not in the poi dataset
柳

### 2019

In [40]:
geo_city_list = set(geomaster_selected["地级"])

for i,MID_city in enumerate(MID_list):
    city = city_list[i]
    csv_file = "summary_2019-V2.csv"

    file_path = os.path.join(city_path,f"{MID_city}_{city}",f"2019_{MID_city}_{city}.csv")
    if os.path.exists(file_path):
        continue
        
    poi_file_list = find_poi_file(MID_city,csv_file)
    
    # wrong file
    if '2019POI-V2/300000/d_320300.csv' in poi_file_list:
        poi_file_list.remove('2019POI-V2/300000/d_320300.csv')
    
    if len(poi_file_list)>0:
        print(city,poi_file_list)        

        # Read the file with the determined encoding
        with open(poi_file_list[0], 'r', encoding='GB2312', errors='ignore') as f:
            first_line = f.readline()
            city_first = first_line.split(",")[5].split(";")[0]
            encoding = 'GB2312' if city_first in geo_city_list else 'utf-8'
        poi_df = pd.read_csv(poi_file_list[0], encoding=encoding,encoding_errors='ignore', on_bad_lines='skip', header=None)        
        
        class_split_columns = poi_df[6].str.split(';', expand=True)
        poi_df["class1"] = class_split_columns[0]
        poi_df["name"] = poi_df[1]
        poi_df["address"] = poi_df[2]
        poi_df["county"] = poi_df[5]
        poi_df["lat"] = poi_df[3]
        poi_df["lon"] = poi_df[4]
        poi_df["class"] = poi_df[6]
        
        poi_df = poi_df[["name","address","county","class","class1","lat","lon"]]
        #print(poi_df.columns)

        

        directory = os.path.dirname(file_path)
        if not os.path.exists(directory):
            # Create the directory
            os.makedirs(directory)

        poi_df.to_csv(file_path,encoding = "utf-8-sig",index = False)
    

深圳市 ['2019POI-V2/400000/d_440300.csv']


In [145]:
poi_df_sub = pd.read_csv(poi_file_list[0],encoding='utf-8', header=None)
city_split_columns = poi_df_sub[5].str.split(';', expand=True)
poi_df_sub[city_split_columns[0] == "深圳市"]

#poi_df_sub.head(5)
#poi_df_sub.loc[poi_df_sub[]]

,0,1,2,3,4,5,6,7
434197,434198,深圳志德建设有限公司睢宁八里商业城工程项目部,324省道南50米,33.903182,117.986409,深圳市;睢宁县,公司企业;公司企业,


### 2018

In [62]:
csv_file = "summary_2018.csv"
year_summary_df = pd.read_csv(save_path + csv_file)
poi_df_list = []
poi_file_list = []

for index, row in tqdm(year_summary_df.iterrows(), total=year_summary_df.shape[0], desc="Processing rows"):
    poi_files = row["path"]
    poi_files  = poi_files.split(";")
    
    for poi_file in poi_files:
        if not poi_file in poi_file_list:
            try:    
                poi_df = pd.read_csv(poi_file, encoding='GB2312', encoding_errors='ignore', on_bad_lines='skip')
                poi_df = poi_df[['address', 'adname','pname', 'cityname','name', 'location',  'type']]
            except Exception as e:
                poi_df = pd.read_csv(poi_file, encoding='GB2312', encoding_errors='ignore', on_bad_lines='skip',quoting=csv.QUOTE_NONE) 
                poi_df = poi_df[['address', 'adname','pname', 'cityname','name', 'location',  'type']]
                
            if poi_df.shape[0]>0:
                poi_file_list.append(poi_file)
                poi_df_list.append(poi_df)

Processing rows: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 373/373 [03:58<00:00,  1.56it/s]


In [66]:
poi_file_list_all = poi_file_list.copy()
poi_file_list_all

['2018POI/1540881779063.csv',
 '2018POI/1540880609855.csv',
 '2018POI/1540884469528.csv',
 '2018POI/1540884533008.csv',
 '2018POI/1540881041983.csv',
 '2018POI/1540883510706.csv',
 '2018POI/1540883124295.csv',
 '2018POI/1540880963716.csv',
 '2018POI/1540881268076.csv',
 '2018POI/1540882992595.csv',
 '2018POI/1540881714158.csv',
 '2018POI/1540882090414.csv',
 '2018POI/1540883057418.csv',
 '2018POI/1540883383324.csv',
 '2018POI/1540880282990.csv',
 '2018POI/1540881343817.csv',
 '2018POI/1540882150995.csv',
 '2018POI/1540881839473.csv',
 '2018POI/1540882536996.csv',
 '2018POI/1540882864204.csv',
 '2018POI/1540883447322.csv',
 '2018POI/1540882470438.csv',
 '2018POI/1540882669138.csv',
 '2018POI/1540882026571.csv',
 '2018POI/1540881964495.csv',
 '2018POI/1540883253652.csv',
 '2018POI/1540883951801.csv',
 '2018POI/1540882601344.csv',
 '2018POI/1540882216803.csv',
 '2018POI/1540882343705.csv',
 '2018POI/1540881585659.csv',
 '2018POI/1540881118637.csv',
 '2018POI/1540883575020.csv',
 '2018POI/

In [64]:
poi_df_list[6]

,address,adname,pname,cityname,name,location,type
0,前兴路与日新东路交叉口东200米,西山区,云南省,昆明市,皇家领海,102.710356，24.998228,购物服务;综合市场;水产海鲜市场
1,龙山路与302国道交叉口西南100米,龙潭区,吉林省,吉林市,顺鑫超市,126.639270，43.892321,购物服务;便民商店/便利店;便民商店/便利店
2,北戴河区,北戴河区,河北省,秦皇岛市,南二街与戴河新路交叉口,119.424920，39.860299,地名地址信息;交通地名;路口名
3,川姜镇志浩村川叠路1号,通州区,江苏省,南通市,美味印象(志浩店),121.067513，31.964380,餐饮服务;糕饼店;糕饼店
4,涧河路与解放北路交叉口东北50米,尖草坪区,山西省,太原市,LC龙城停车场,112.563126，37.914226,交通设施服务;停车场;公共停车场
...,...,...,...,...,...,...,...
999995,人民路与风华路交叉口东50米,樊城区,湖北省,襄阳市,壹久行烟酒直供超市,112.139368，32.040591,购物服务;专卖店;烟酒专卖店|购物服务;便民商店/便利店;便民商店/便利店
999996,里水镇河村江边南路8号,南海区,广东省,佛山市,佛山市华谋气动食品机械有限公司,113.166500，23.165965,公司企业;公司;机械电子
999997,北航路中医院红绿灯上行200米,西秀区,贵州省,安顺市,世通小吃,105.917037，26.258230,餐饮服务;快餐厅;快餐厅
999998,李口镇李口初级中学附近,睢阳区,河南省,商丘市,紫罗兰家纺,115.581134，34.250116,购物服务;家居建材市场;布艺市场


In [67]:
for i,MID_city in enumerate(MID_list):
    city = city_list[i]
    csv_file = "summary_2018.csv"

    file_path = os.path.join(city_path,f"{MID_city}_{city}",f"2018_{MID_city}_{city}.csv")
    if os.path.exists(file_path):
        continue
        
    poi_file_list = find_poi_file(MID_city,csv_file)
    
    if len(poi_file_list)>0:
        print(city,len(poi_file_list))
        
        # read the first df as base
        index_0 = poi_file_list_all.index(poi_file_list[0])
        poi_df = poi_df_list[index_0]
        city_df_all = poi_df.copy(deep=False)
        city_df_all.drop(city_df_all.index, inplace=True)                    

        # Read the rest files
        for file in tqdm(poi_file_list):
            index = poi_file_list_all.index(file)
            poi_df = poi_df_list[index]
            # find parts that belong to this city
            city_df = poi_df.loc[poi_df['cityname'] == city]
            city_df_all = pd.concat([city_df_all, city_df])
            

        class_split_columns = city_df_all["type"].str.split(';', expand=True)
        city_df_all["class1"] = class_split_columns[0]
        coord_split_columns =city_df_all["location"].str.split('，', expand=True)
        city_df_all["lat"] = coord_split_columns[1]
        city_df_all["lon"] = coord_split_columns[0]
        city_df_all.head()
        
        directory = os.path.dirname(file_path)
        if not os.path.exists(directory):
            # Create the directory
            os.makedirs(directory)

        city_df_all.to_csv(file_path,encoding = "utf-8-sig",index = False)


兴安盟 66


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 66/66 [00:03<00:00, 18.93it/s]


克孜勒苏柯尔克孜自治州 61


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 61/61 [00:02<00:00, 25.90it/s]


酒泉市 61


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 58/58 [00:02<00:00, 21.52it/s]


莱芜市 66


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 66/66 [00:03<00:00, 20.56it/s]


那曲地区 49


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 49/49 [00:01<00:00, 25.19it/s]
